In [1]:
import pandas as pd 
import glob

In [15]:
download_dates = pd.date_range(start='20200403', end='20210307').strftime('%Y%m%d')

In [16]:
download_dates

Index(['20200403', '20200404', '20200405', '20200406', '20200407', '20200408',
       '20200409', '20200410', '20200411', '20200412',
       ...
       '20210226', '20210227', '20210228', '20210301', '20210302', '20210303',
       '20210304', '20210305', '20210306', '20210307'],
      dtype='object', length=339)

In [17]:
base_url = ("https://www.lgl.bayern.de/gesundheit/infektionsschutz/infektionskrankheiten_a_z/coronavirus/karte_coronavirus/fallzahlen_archiv/") #  add format option to year 

In [18]:
base_url

'https://www.lgl.bayern.de/gesundheit/infektionsschutz/infektionskrankheiten_a_z/coronavirus/karte_coronavirus/fallzahlen_archiv/'

In [19]:
#  create list of remote URL from base URL
list_of_url = [base_url+date+str('_LK_coronazahlen.csv') for date in download_dates]


In [14]:
list_of_url;

In [22]:
#  download and combine multiple files into one DataFrame
df = pd.DataFrame()
for url in list_of_url:
    try:tempdf = pd.read_csv(url, encoding='latin-1',delimiter=';')
        df = pd.concat(tempdf)
    except: print(str('no data for')+url)




IndentationError: unexpected indent (<ipython-input-22-53d75c2af647>, line 5)

In [ ]:
https://www.lgl.bayern.de/gesundheit/infektionsschutz/infektionskrankheiten_a_z/coronavirus/karte_coronavirus/fallzahlen_archiv/20210307.csv

In [24]:
df = pd.read_csv('https://www.lgl.bayern.de/gesundheit/infektionsschutz/infektionskrankheiten_a_z/coronavirus/karte_coronavirus/fallzahlen_archiv/20200404_LK_coronazahlen.csv', encoding='latin-1',delimiter=';')

In [26]:
df['Date'] = '20200404'

In [27]:
df.head()

,Regierungsbezirk Land-/Stadtkreis,Anzahl Fälle,Änderung zum Vortag Fälle,Anzahl Todesfälle,Änderung zum Vortag Todesfälle,Population,Fälle/100.000 kumulativ,Fälle der letzten 7 Tage,7-Tages-Inzidenz,Date
0,LK Aichach-Friedberg,164,10,3,0,133596,"122,76",68,"50,9",20200404
1,LK Altötting,233,29,6,1,111210,"209,51",138,"124,09",20200404
2,LK Amberg-Sulzbach,142,21,3,0,103109,"137,72",86,"83,41",20200404
3,LK Ansbach,250,13,1,1,183949,"135,91",130,"70,67",20200404
4,LK Aschaffenburg,210,23,12,2,174208,"120,55",94,"53,96",20200404


In [27]:
df = df.loc[df['Regierungsbezirk Land-/Stadtkreis'] == 'SK Nürnberg']

In [28]:
df

,Regierungsbezirk Land-/Stadtkreis,Anzahl Fälle,Änderung zum Vortag Fälle,Anzahl Todesfälle,Änderung zum Vortag Todesfälle,Population,Fälle/100.000 kumulativ,Fälle der letzten 7 Tage,7-Tages-Inzidenz
87,SK Nürnberg,24144,79,800,0,518370,"4657,68",427,"82,37"
